# MMA 662 - Assignment 2
## Instructions:
1. Answer the questions in this Jupyter Notebook file and upload it in the Assignment 2 entry under the Assignments tab in MyCourses. Answer each question in its own place. Late submissions are NOT accepted. Also, files not in the correct format (other than a Jupyter Notebook) are NOT accepted.
2. You can submit your answer file multiple times, but only the last submission is kept and graded.
3. For each question, make sure to print all the results that the question asks. Also, prevent printing unnecessary information.
4. We only grade the final answer, not the code, for coding questions. However, as the coding part of Question 7 can be a bit challenging, we will give **partial credit** for **concise and correct** explanations provided instead of the code only for this question. (Simple criterion: Your explanations should shed light on things that are not obvious from the question itself!)
5. Simple explanations are not enough for formulation questions, and you have to write the exact mathematical formulation of the problem. If you are using an additional variable, introduce it clearly.

In [2]:
!pip install -q gurobipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 91.6 MB/s eta 0:00:00


In [4]:
# --- Activate Gurobi Academic (WLS) in Colab ---
!pip install -q gurobipy

# Create the license file in Colab (ephemeral each session)
license_text = """# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repositories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID=3b0100a9-3166-4003-8b5a-dd4c80226c63
WLSSECRET=f7be1a22-8281-423f-ab1d-1244842a6ad6
LICENSEID=2718149
"""
with open('/root/gurobi.lic', 'w') as f:
    f.write(license_text)

import os
os.environ['GRB_LICENSE_FILE'] = '/root/gurobi.lic'



In [5]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: The Apportionment Problem

## The Story:
The problem of allocating parliamentary seats in a *fair* way is a political matter in democratic societies. Suppose a country with a total population of $N = \sum_i p_i$ tries to allocate its $s$ available seats to its provinces, each with a population of $p_i$. The *common sense* says that the *fair* way to distribute these seats is to allocate each province a seat quota proportional to its population:
$$q_i = s \times \frac{p_i}{\sum_i p_i}.$$

Although this solution seems appealing, it ignores that the number of allocated seats must be **integers**. For instance, in a country with 30 million population and 130 parliament seats, the seat quota for a province with a population of 5 million is $q_i = \frac{5 \times 130}{30} = 21.\overline{66}$, which is impossible. In this problem, we will use two methods that you are familiar with to address this issue - plus some additional considerations.

## The Data:
The country we are working on has 20 provinces, and its parliament has 601 seats. In the attached file *assignment2_data.csv*, you can find the dataset for this problem. The first column is the province's name, the second column is the province's population, and the third column is the quota ($q_i$) for the province. The quota for each province is the continuous value we calculated using the formula above as $q_i = s \times \frac{p_i}{\sum_i p_i}$. Read the dataset using Python's Pandas package - or any package you choose - and extract the relevant information. (To ensure everyone is working with the same data, do not calculate the quotas again yourself; just use the given $q_i$ s.) For example, in this dataset, the population of province $P4$ is 3200000, and the quota for this province is 13.16359.

## Question 1: The Min Sum Approach: (7.5 points)

You have seen this approach in your first quiz. In this approach, the goal is to minimize the sum of absolute gaps between the number of allocated seats to each province ($a_i$) and its quota ($q_i$) while leaving no seats unfilled. That is, we seek to find allocations $a_0$ to $a_{19}$ to
$$\min_{a_0, \dots, a_{19}} \sum_{i=0}^{19} |q_i - a_i| \\
   s.t. \qquad \sum_{i=0}^{19} a_i = s \\
   a_i \in Z^+ \: for \: i = 0, \dots, 19.$$
(The last line means the number of allocated seats to each province has to be a non-negative integer.)

Linearize the above problem using the method you learned in the quiz, write its code using Gurobi, and solve it. Report the sum of absolute gaps for the optimal solution and the number of seats allocated to each province.

(Hint: notice that when you are linearizing the absolute value, the auxiliary variables you must introduce should be continuous, not discrete. This is because while the allocations are discrete, the gap between allocations and the quotas is continuous.)

# Mathematical Formulation

## Decision Variables
- $a_i \in \mathbb{Z}^+$ = number of seats allocated to province $i$ (for $i = 0, 1, \dots, 19$)
- $g_i^+ \in \mathbb{R}^+$ = positive gap for province $i$ (continuous auxiliary variable)
- $g_i^- \in \mathbb{R}^+$ = negative gap for province $i$ (continuous auxiliary variable)

## Objective Function
$$
\min \sum_{i=0}^{19} (g_i^+ + g_i^-)
$$

## Constraints

### Gap Linearization
$$
a_i - q_i = g_i^+ - g_i^- \quad \forall i
$$
This ensures:
$$
|q_i - a_i| = g_i^+ + g_i^- \quad \text{(absolute value linearization)}
$$

### Total Seats
$$
\sum_{i=0}^{19} a_i = 601
$$

### Non-negativity
$$
a_i \geq 0 \quad \text{and integer}, \quad g_i^+, g_i^- \geq 0
$$

---

## Insight on Linearization
The absolute value $|q_i - a_i|$ is replaced by decomposing the gap into positive and negative parts:

- $g_i^+$ captures the **excess** (when $a_i > q_i$)
- $g_i^-$ captures the **shortfall** (when $a_i < q_i$)

At optimality, $g_i^+$ and $g_i^-$ **will not both be positive simultaneously**.

---
- The model **minimizes the total absolute deviation** between each province’s allocated seats $a_i$ and its ideal quota $q_i$, producing the **fairest overall seat distribution** in the **L₁ (sum of absolute deviations)** sense.

- The decomposition  
  $$
  a_i - q_i = g_i^+ - g_i^-
  $$  
  **guarantees linearity** while preserving the true magnitude of deviation via  
  $$
  |a_i - q_i| = g_i^+ + g_i^-.
  $$

- **At optimality**, for every province $i$, **at most one** of $g_i^+$ or $g_i^-$ is positive — ensuring the **correct absolute-value representation** without overlap.

- The **single equality constraint**  
  $$
  \sum_i a_i = 601
  $$  
  **enforces exact distribution** of all 601 seats — no fractions, no surplus, no shortfall.

- Since all coefficients are **rational** and the constraint matrix is **totally unimodular** (or integrality is preserved), the **optimal solution is automatically integer** — **no rounding required**.

In [7]:
# Read data from the csv file

df = pd.read_csv('assignment2_data.csv')
Province = df['Province'].tolist()
Population = df['Population'].tolist()
q = df['Quota'].tolist()

Num = len(Province)
# Total seats:

s = 601

###############################################

# Create a new model
model = gb.Model("Question1")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:

# Decision variables
# a[i] = integer seats allocated to province i
a = model.addVars(Num, vtype=gb.GRB.INTEGER, lb=0, name="seats")

# Auxiliary variables for gap linearization (CONTINUOUS, not integer)
# g_pos[i] = positive gap (when a[i] > q[i])
# g_neg[i] = negative gap (when a[i] < q[i])
g_pos = model.addVars(Num, vtype=gb.GRB.CONTINUOUS, lb=0, name="gap_pos")
g_neg = model.addVars(Num, vtype=gb.GRB.CONTINUOUS, lb=0, name="gap_neg")

# Objective: minimize sum of absolute gaps
# |q[i] - a[i]| = g_pos[i] + g_neg[i]
model.setObjective(gb.quicksum(g_pos[i] + g_neg[i] for i in range(Num)), gb.GRB.MINIMIZE)

# Constraints
# 1. Gap linearization: a[i] - q[i] = g_pos[i] - g_neg[i]
# Rearranged: a[i] = q[i] + g_pos[i] - g_neg[i]
for i in range(Num):
    model.addConstr(a[i] - g_pos[i] + g_neg[i] == q[i], name=f"gap_{i}")

# 2. Total seats constraint
model.addConstr(gb.quicksum(a[i] for i in range(Num)) == s, name="total_seats")

# Optimize
model.optimize()

# Extract and print results
print("=" * 60)
print("QUESTION 1: MIN SUM APPROACH")
print("=" * 60)

if model.status == gb.GRB.OPTIMAL:
    print(f"\nOptimal Sum of Absolute Gaps: {model.ObjVal:.6f}\n")
    print(f"{'Province':<10} {'Quota':<12} {'Allocated':<12} {'Gap':<12}")
    print("-" * 50)

    total_allocated = 0
    for i in range(Num):
        allocated = int(a[i].X)
        gap = abs(q[i] - allocated)
        total_allocated += allocated
        print(f"{Province[i]:<10} {q[i]:<12.5f} {allocated:<12} {gap:<12.6f}")

    print("-" * 50)
    print(f"{'TOTAL':<10} {'':<12} {total_allocated:<12}")
else:
    print("Model did not converge to optimality")

Set parameter MIPGap to value 1e-05
QUESTION 1: MIN SUM APPROACH

Optimal Sum of Absolute Gaps: 4.616000

Province   Quota        Allocated    Gap         
--------------------------------------------------
P0         21.80219     22           0.197810    
P1         37.84531     38           0.154690    
P2         22.21355     22           0.213550    
P3         32.90897     33           0.091030    
P4         13.16359     13           0.163590    
P5         36.19986     36           0.199860    
P6         33.32033     33           0.320330    
P7         41.54757     42           0.452430    
P8         14.80903     15           0.190970    
P9         25.50445     25           0.504450    
P10        48.12936     48           0.129360    
P11        23.85900     24           0.141000    
P12        28.79535     29           0.204650    
P13        25.91581     26           0.084190    
P14        19.74538     20           0.254620    
P15        44.01574     44           0.0157

# Explanation of Implementation

## Key Points

1. **Gap Linearization**  
   The absolute value $|q_i - a_i|$ is linearized by introducing two **continuous non-negative variables**:

   - $g_{\text{pos}}[i]$ captures $a_i - q_i$ when $a_i > q_i$  
   - $g_{\text{neg}}[i]$ captures $q_i - a_i$ when $a_i < q_i$

   The constraint:
   $$
   a_i - g_{\text{pos}}[i] + g_{\text{neg}}[i] = q_i
   $$
   ensures **only one** of $g_{\text{pos}}[i]$ or $g_{\text{neg}}[i]$ is active at a time.  
   The objective minimizes their sum, which equals the absolute gap:
   $$
   |q_i - a_i| = g_{\text{pos}}[i] + g_{\text{neg}}[i]
   $$

2. **Why Continuous Gaps?**  
   Although seat allocations $a_i$ must be **integers**, the quotas $q_i$ are typically **non-integer** (e.g., based on population proportions).  
   The resulting **gaps** ($q_i - a_i$) are naturally **continuous**.  
   Using continuous gap variables gives **Gurobi more flexibility** during optimization and avoids unnecessary integer constraints.

3. **Total Seats Constraint**  
   Ensures **exactly 601 seats** are distributed  no shortfall, no excess.

## Question 2: The Min Max Approach: (7.5 points)

You are familiar with this approach too! Here, instead of the sum of absolute gaps, we want to minimize the absolute value of the largest gap while leaving no seats unfilled. In other words, we want to find the allocations $a_0$ to $a_{19}$ to:
$$\min_{a_0, \dots, a_{19}} \max_{i} |q_i - a_i| \\
   s.t. \qquad \sum_{i=0}^{19} a_i = s \\
   a_i \in Z^+ \: for \: i = 0, \dots, 19.$$

Linearize the above problem using the method you learned in the quiz, write its code using Gurobi, and solve it. Report the maximum absolute gap in the optimal solution, the number of seats allocated to each province, and the sum of absolute gaps in the optimal solution.

# QUESTION 2: Min Max Approach (7.5 points)

## Objective
Minimize the **maximum absolute deviation** (minimax criterion), ensuring the largest gap is as small as possible.

## Mathematical Formulation

### Decision Variables
- $a_i \in \mathbb{Z}^+$ = seats allocated to province $i$ (for $i = 0, 1, \dots, 19$)
- $z \in \mathbb{R}^+$ = maximum absolute gap (auxiliary variable)

### Objective Function
$$
\min z
$$

z is continious value
### Constraints

1. **Gap Upper Bounds**  
   $$
   |q_i - a_i| \leq z \quad \forall i
   $$  
   *Equivalently:*  
   $$
   a_i - q_i \leq z \quad \text{and} \quad q_i - a_i \leq z \quad \forall i
   $$

2. **Total Seats**  
   $$
   \sum_{i=0}^{19} a_i = 601
   $$

3. **Non-negativity and Integrality**  
   $$
   a_i \geq 0 \quad \text{and integer}, \quad z \geq 0
   $$

---

## Key Difference from Min Sum

- **Min Sum**: Balances fairness across all provinces; some may have large gaps if it reduces the *total* deviation.
- **Min Max**: Prioritizes **equity**; ensures **no province** is too severely underrepresented or overrepresented.

In [8]:
# Read data (same as Question 1)
df = pd.read_csv('assignment2_data.csv')
Province = df['Province'].tolist()
Population = df['Population'].tolist()
q = df['Quota'].tolist()

Num = len(Province)
s = 601

# Create model
model = gb.Model("Question2")
model.setParam('MIPGap', 0.00001)
model.Params.OutputFlag = 0

# Decision variables
a = model.addVars(Num, vtype=gb.GRB.INTEGER, lb=0, name="seats")

# Auxiliary variable: maximum absolute gap
z = model.addVar(vtype=gb.GRB.CONTINUOUS, lb=0, name="max_gap")

# Objective: minimize maximum gap
model.setObjective(z, gb.GRB.MINIMIZE)

# Constraints
# 1. Gap upper bounds: |q[i] - a[i]| <= z
# This is equivalent to: a[i] - q[i] <= z AND q[i] - a[i] <= z
for i in range(Num):
    model.addConstr(a[i] - q[i] <= z, name=f"gap_upper_pos_{i}")
    model.addConstr(q[i] - a[i] <= z, name=f"gap_upper_neg_{i}")

# 2. Total seats constraint
model.addConstr(gb.quicksum(a[i] for i in range(Num)) == s, name="total_seats")

# Optimize
model.optimize()

# Extract and print results
print("=" * 60)
print("QUESTION 2: MIN MAX APPROACH")
print("=" * 60)

if model.status == gb.GRB.OPTIMAL:
    max_gap = z.X
    print(f"\nOptimal Maximum Absolute Gap: {max_gap:.6f}\n")

    print(f"{'Province':<10} {'Quota':<12} {'Allocated':<12} {'Gap':<12}")
    print("-" * 50)

    total_allocated = 0
    total_sum_gaps = 0
    for i in range(Num):
        allocated = int(a[i].X)
        gap = abs(q[i] - allocated)
        total_allocated += allocated
        total_sum_gaps += gap
        print(f"{Province[i]:<10} {q[i]:<12.5f} {allocated:<12} {gap:<12.6f}")

    print("-" * 50)
    print(f"{'TOTAL':<10} {'':<12} {total_allocated:<12}")
    print(f"\nSum of Absolute Gaps: {total_sum_gaps:.6f}")
else:
    print("Model did not converge to optimality")

Set parameter MIPGap to value 1e-05
QUESTION 2: MIN MAX APPROACH

Optimal Maximum Absolute Gap: 0.504450

Province   Quota        Allocated    Gap         
--------------------------------------------------
P0         21.80219     22           0.197810    
P1         37.84531     38           0.154690    
P2         22.21355     22           0.213550    
P3         32.90897     33           0.091030    
P4         13.16359     13           0.163590    
P5         36.19986     36           0.199860    
P6         33.32033     33           0.320330    
P7         41.54757     42           0.452430    
P8         14.80903     15           0.190970    
P9         25.50445     25           0.504450    
P10        48.12936     48           0.129360    
P11        23.85900     24           0.141000    
P12        28.79535     29           0.204650    
P13        25.91581     26           0.084190    
P14        19.74538     20           0.254620    
P15        44.01574     44           0.0157

**Interpretation of Output**

The maximum absolute deviation across all provinces is 0.50445, meaning no

province’s seat allocation differs from its quota by more than half a seat.

All 601 seats are exactly allocated, satisfying the total-seat constraint.

The sum of absolute gaps (4.6160) is slightly higher than in Q1, which is expected  the Min–Max model prioritizes equity of deviation (uniform fairness) rather than minimizing total deviation.

Compared to the Min–Sum result, the solution slightly redistributes seats among provinces to balance extreme over/underrepresentation, achieving a tighter maximum gap but a marginally larger total error.

## Question 3: Political Considerations: (20 points)

Due to political reasons, some considerations should be addressed when allocating seats to each province. Here, we want to implement three of them:
1. The number of seats allocated to the two least populated provinces ($P4 \ \text{and} \ P8$) must be equal.
2. The number of seats allocated to each province with more than 10,000,000 population ($P7, P10, \ \text{and} \ P15$) should not exceed its quota.
3. If the number of seats allocated to $P0$ is more than its quota, then the number of seats allocated to $P2$ must be more than its quota as well.

Now answer the following questions:

### Question 3.1: Mathematical Formulation of the Considerations:

Write the mathematical formulation for each of the above considerations using the techniques you've learned in the course in the course in the following Markdown cell. (Your writing should be clear about which formula is for which consideration.) (10 points)

# Q3.1 – Political Considerations (Exact Formulations)

Let $a_i \in \mathbb{Z}_{\geq 0}$ be seats for province $i$, and quotas $q_i \in \mathbb{R}_{> 0}$.

---
### First Constraint:

### 1. Equal Seats for the Two Least-Populated Provinces (P4, P8)
$$
a_{\text{P4}} = a_{\text{P8}}
$$

---
### Second Constraint:
### 2. No Province with Population > 10,000,000 Exceeds Its Quota  
Applies to **P7, P10, P15**:
$$
a_i \leq q_i \quad \text{for } i \in \{\text{P7}, \text{P10}, \text{P15}\}
$$
 *(Since $a_i$ is integer, this is equivalent to $a_i \leq \lfloor q_i \rfloor$.)*

---
### Third Constraint:
### 3. Implication: "If P0 is above quota, then P2 must be above quota"

Define:  
$$
Q_0 = \lceil q_{\text{P0}} \rceil, \quad Q_2 = \lceil q_{\text{P2}} \rceil
$$

Introduce **binary variables**:  
$$
y_0, y_2 \in \{0,1\}
$$

With **big-M** ($M$ large, e.g., $M = 601$):

$$
\begin{aligned}
& a_{\text{P0}} \geq Q_0 - M(1 - y_0) \\
& a_{\text{P0}} \leq (Q_0 - 1) + M y_0 \\
& a_{\text{P2}} \geq Q_2 - M(1 - y_2) \\
& a_{\text{P2}} \leq (Q_2 - 1) + M y_2 \\
& y_0 \leq y_2
\end{aligned}
$$

---

## Logical Effect
This enforces:  
$$
a_{\text{P0}} \geq Q_0 \quad \Rightarrow \quad a_{\text{P2}} \geq Q_2
$$

i.e., **if P0 gets at least its ceiling quota, then P2 must also get at least its ceiling quota.**

### Question 3.2: Implementing the Considerations:

Based on the first approach (minimizing the sum of absolute gaps while filling all the seats) and by adding these considerations as new constraints. Find the sum of absolute gaps for the optimal solution using Gurobi and report it. (10 points)

In [32]:
# Read data
df = pd.read_csv('assignment2_data.csv')
Province = df['Province'].tolist()
Population = df['Population'].tolist()
q = df['Quota'].tolist()

Num = len(Province)
s = 601

import math
import gurobipy as gb
from gurobipy import GRB

# Indices (robust, no magic numbers)
idx = {Province[i]: i for i in range(Num)}
iP0, iP2 = idx['P0'], idx['P2']
iP4, iP8 = idx['P4'], idx['P8']
iP7, iP10, iP15 = idx['P7'], idx['P10'], idx['P15']

Q0 = math.ceil(q[iP0])
Q2 = math.ceil(q[iP2])
M  = s  # safe big-M

# Model
model = gb.Model("Question3")
model.Params.OutputFlag = 0
model.Params.MIPGap = 1e-9
model.Params.FeasibilityTol = 1e-9
model.Params.IntFeasTol = 1e-9

# Decision variables
a = model.addVars(Num, vtype=GRB.INTEGER, lb=0, name="seats")

# Absolute-gap linearization (Q1)
g_pos = model.addVars(Num, vtype=GRB.CONTINUOUS, lb=0, name="gap_pos")
g_neg = model.addVars(Num, vtype=GRB.CONTINUOUS, lb=0, name="gap_neg")
for i in range(Num):
    model.addConstr(a[i] - g_pos[i] + g_neg[i] == q[i])

# Total seats
model.addConstr(gb.quicksum(a[i] for i in range(Num)) == s)

# Objective: minimize total absolute deviation
model.setObjective(gb.quicksum(g_pos[i] + g_neg[i] for i in range(Num)), GRB.MINIMIZE)

# --- Political constraints ---

# (1) Equal seats for P4 and P8
model.addConstr(a[iP4] == a[iP8])

# (2) Large provinces cannot exceed quota
model.addConstr(a[iP7]  <= q[iP7])
model.addConstr(a[iP10] <= q[iP10])
model.addConstr(a[iP15] <= q[iP15])

# (3) Implication: if P0 is at/above ceil(q0), then P2 is at/above ceil(q2)
y0 = model.addVar(vtype=GRB.BINARY, name="y0")  # 1 ⇔ a[P0] ≥ Q0
y2 = model.addVar(vtype=GRB.BINARY, name="y2")  # 1 ⇔ a[P2] ≥ Q2

# Bind y0 to the threshold on a[P0]
model.addConstr(a[iP0] >= Q0 - M*(1 - y0))
model.addConstr(a[iP0] <= (Q0 - 1) + M*y0)

# Bind y2 to the threshold on a[P2]
model.addConstr(a[iP2] >= Q2 - M*(1 - y2))
model.addConstr(a[iP2] <= (Q2 - 1) + M*y2)

# Enforce implication y0 ⇒ y2
model.addConstr(y0 <= y2)

# Solve
model.optimize()

# Report
print("="*60)
print("QUESTION 3: POLITICAL CONSIDERATIONS (Min-Sum)")
print("="*60)
print(f"\nOptimal Sum of Absolute Gaps: {model.ObjVal:.6f}\n")
print(f"{'Province':<10} {'Quota':<12} {'Allocated':<12} {'Gap':<12}")
print("-"*50)
total_alloc = 0
for i in range(Num):
    alloc = int(a[i].X)
    total_alloc += alloc
    print(f"{Province[i]:<10} {q[i]:<12.5f} {alloc:<12} {abs(q[i]-alloc):<12.6f}")
print("-"*50)
print(f"TOTAL {'':<12} {total_alloc:<12}")

# Validation
print("\nValidation of Political Constraints:")
print(f"  P4 vs P8 equal? {int(a[iP4].X)} == {int(a[iP8].X)}")
print(f"  P7: {int(a[iP7].X)} <= {q[iP7]:.5f} → {int(a[iP7].X) <= q[iP7]}")
print(f"  P10: {int(a[iP10].X)} <= {q[iP10]:.5f} → {int(a[iP10].X) <= q[iP10]}")
print(f"  P15: {int(a[iP15].X)} <= {q[iP15]:.5f} → {int(a[iP15].X) <= q[iP15]}")
print(f"  Implication holds? y0={int(y0.X)}, y2={int(y2.X)}, "
      f"a[P0]={int(a[iP0].X)} (≥{Q0}? {int(a[iP0].X) >= Q0}), "
      f"a[P2]={int(a[iP2].X)} (≥{Q2}? {int(a[iP2].X) >= Q2})")


QUESTION 3: POLITICAL CONSIDERATIONS (Min-Sum)

Optimal Sum of Absolute Gaps: 6.574920

Province   Quota        Allocated    Gap         
--------------------------------------------------
P0         21.80219     22           0.197810    
P1         37.84531     38           0.154690    
P2         22.21355     23           0.786450    
P3         32.90897     33           0.091030    
P4         13.16359     14           0.836410    
P5         36.19986     36           0.199860    
P6         33.32033     33           0.320330    
P7         41.54757     41           0.547570    
P8         14.80903     14           0.809030    
P9         25.50445     25           0.504450    
P10        48.12936     48           0.129360    
P11        23.85900     24           0.141000    
P12        28.79535     29           0.204650    
P13        25.91581     26           0.084190    
P14        19.74538     20           0.254620    
P15        44.01574     44           0.015740    
P16        

## Interpretation of Output – Political Constraints (Min-Sum Model)

- **Objective Impact**:  
  The optimal total deviation **increased from 4.616 to 6.5749**.  
  This rise reflects the **tightened feasible region** due to added **political and fairness constraints**.  
  → A higher objective value indicates that **political balance comes at a cost to numerical fairness**.

---

- **Constraint Verification**:

  - **Equal Seats (P4 = P8)**:  
    Both provinces received **14 seats** → **Equality rule satisfied**.

  - **No-Exceed Rule (Large Provinces)**:  
    - P7: 41 ≤ quota  
    - P10: 48 ≤ quota  
    - P15: 44 ≤ quota  
    → All allocations **at or below quota** → **Compliance confirmed**.

  - **Conditional Rule (If P0 ≥ ceiling, then P2 ≥ ceiling)**:  
    - $ \lceil q_0 \rceil = \lceil 21.80 \rceil = 22 $  
    - $ a_0 = 22 \geq 22 $ → condition triggered  
    - Binary indicators: $ (y_0, y_2) = (1, 1) $  
    - $ \lceil q_2 \rceil = \lceil 22.21 \rceil = 23 $  
    - $ a_2 = 23 \geq 23 $ → **implication enforced correctly**

→ **All political constraints are active and satisfied** in the optimal solution.

# Problem 2: Designing the Floor Plan

## The Story and Data:

You are responsible for designing the floor plan of a shopping mall. The management team wants to include stores and facilities that maximize its expected annual profit while satisfying some constraints. In the following table, called *the original table*, you can see the yearly profit expected from one unit of each type of store/facility, the area it occupies, and the maximum number of units you can build from each type. (Notice that the mall can have more than one unit of each kind of store/facility as long as you do not pass the maximum number limit.)

| Facility/Store | Expected Annual Profit (\$) | Occupied Area ($m^2$) | Max Number of Units |
| :--: | :--: | :--: | :--: |
| Restaurant | 300,000 | 500 | 5 |
| Clothing Store | 800,000 | 1,700 | 15 |
| Tech Store | 1,000,000 | 1,000 | 5 |
| Pharmacy | 500,000 | 900 | 5 |
| Department Store | 2,200,000 | 4,300 | 3 |
| Toy Store | 500,000 | 1,400 | 5 |
| Toilet | -25,000 | 200 | 3 |
| Security Division | -100,000 | 300 | 3 |

There are two fundamental constraints that you need to consider when designing the floor plan **in all questions**:

1. The total area of all stores and facilities should be at most 20,000 $m^2$. (Just so you know, Montreal's Eaton Center's total retail floor area is 45,000 $m^2$.)
2. The mall should have at least one complete toilet and one complete security division. (By "at least one complete," we mean $\geq 1$.)

For example, we decided to have 2 restaurants, 2 clothing stores, 1 tech store, 3 pharmacies, 3 toy stores, 1 toilet, and 2 security divisions in the mall. The expected annual profit of this floor plan is:
$$2 \times \$300,000 + 2 \times \$800,000 + 1 \times \$1,000,000 + 3 \times \$500,000 + 3 \times \$500,000 - 1 \times \$25,000 - 2 \times \$100,000 = \$5,975,000$$

## Question 4: Using Continuous Variables: (15 points)

Suppose you can have continuous numbers of each type of store/facility. For instance, you are allowed to have 3.45 units of pharmacies in your mall. Remember to consider the two fundamental constraints.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a linear programming problem, solve it, and report the results using Gurobi or by hand. Remember, if you solve it by hand, you must provide solid, convincing explanations.

# QUESTION 4: Continuous Variables

## Mathematical Formulation

### Decision Variables
$$
x_j \in \mathbb{R}^+, \quad j = 0, 1, \dots, 7
$$
where $x_j$ = number of units of facility $j$

---

### Objective Function
$$
\max \sum_{j=0}^{7} p_j \cdot x_j
$$
where $p_j$ = profit per unit of facility $j$

---

### Constraints

1. **Area Constraint**  
   $$
   \sum_{j=0}^{7} a_j \cdot x_j \leq 20,000
   $$

2. **Maximum Units per Facility**  
   $$
   0 \leq x_j \leq M_j \quad \text{for all } j
   $$
   (where $M_j$ = maximum allowable units of facility $j$)

3. **Mandatory Facilities**  
   $$
   x_6 \geq 1 \quad \text{(toilet)} \quad \text{and} \quad x_7 \geq 1 \quad \text{(security division)}
   $$

---

## Solution Strategy

This is a **linear programming (LP)** problem with **continuous variables**.  
It can be solved efficiently using the **Simplex Method**.

- The optimal solution occurs at a **vertex** of the feasible region.
- This LP relaxation provides an **upper bound** on the maximum profit.
- The **integer-constrained version** (if required later) will yield a **potentially lower** (but feasible) profit.

In [35]:

Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)
TAREA   = 20000

# Model
m = gb.Model("Q4_Continuous")
m.Params.OutputFlag = 0

# Decision variables: continuous with caps
x = m.addVars(Num, vtype=GRB.CONTINUOUS, lb=0.0, ub=Max_No, name="x")

# Constraints
m.addConstr(gb.quicksum(Area[j]*x[j] for j in range(Num)) <= TAREA, name="total_area")
m.addConstr(x[6] >= 1, name="min_toilet")    # Toilet
m.addConstr(x[7] >= 1, name="min_security")  # Security

# Objective: maximize profit
m.setObjective(gb.quicksum(Profit[j]*x[j] for j in range(Num)), GRB.MAXIMIZE)

# Optimize
m.optimize()

# ---- Reporting (exact, internally consistent) ----
obj = m.ObjVal
units = [x[j].X for j in range(Num)]
tot_area  = sum(Area[j]*units[j]  for j in range(Num))
tot_profit= sum(Profit[j]*units[j] for j in range(Num))

# Consistency checks (no placeholders)
assert abs(obj - tot_profit) <= 1e-6
assert tot_area <= TAREA + 1e-6
assert all(0.0 - 1e-9 <= units[j] <= Max_No[j] + 1e-9 for j in range(Num))
assert units[6] >= 1 - 1e-9 and units[7] >= 1 - 1e-9

print("="*62)
print("QUESTION 4: CONTINUOUS VARIABLES (LP)")
print("="*62)
print(f"\nMaximum Expected Annual Profit: ${obj:,.2f}\n")
print(f"{'Facility':<16}{'Units':>10}  {'Profit/Unit':>14}  {'Total Profit':>15}")
print("-"*62)
for j in range(Num):
    tp = Profit[j]*units[j]
    print(f"{Type[j]:<16}{units[j]:>10.6f}  ${Profit[j]:>13,.0f}  ${tp:>14,.2f}")
print("-"*62)
print(f"Total Occupied Area: {tot_area:,.2f} m²  (Limit: {TAREA:,} m²)")
print(f"Area Utilization: {tot_area/TAREA*100:,.2f}%")

QUESTION 4: CONTINUOUS VARIABLES (LP)

Maximum Expected Annual Profit: $12,712,209.30

Facility             Units     Profit/Unit     Total Profit
--------------------------------------------------------------
Restaurant        5.000000  $      300,000  $  1,500,000.00
Clothing Store    0.000000  $      800,000  $          0.00
Tech Store        5.000000  $    1,000,000  $  5,000,000.00
Pharmacy          5.000000  $      500,000  $  2,500,000.00
Dept Store        1.744186  $    2,200,000  $  3,837,209.30
Toy Store         0.000000  $      500,000  $          0.00
Toilet            1.000000  $      -25,000  $    -25,000.00
Security          1.000000  $     -100,000  $   -100,000.00
--------------------------------------------------------------
Total Occupied Area: 20,000.00 m²  (Limit: 20,000 m²)
Area Utilization: 100.00%


## Question 5: Using Integer Variables: (10 points)

In realistic settings, the number of stores/facilities should be discrete. Again, we are interested in the maximum expected annual profit and the optimal combination. Remember to consider the two fundamental constraints.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as an integer programming problem, solve it, and report the results using Gurobi.

# QUESTION 5: Integer Variables (10 points)

## Mathematical Formulation

### Decision Variables
$$
x_j \in \mathbb{Z}^+, \quad j = 0, 1, \dots, 7
$$
where $x_j$ = number of units of facility $j$

---

### Objective Function
$$
\max \sum_{j=0}^{7} p_j \cdot x_j
$$
where $p_j$ = profit per unit of facility $j$

---

### Constraints

1. **Area Constraint**  
   $$
   \sum_{j=0}^{7} a_j \cdot x_j \leq 20,000
   $$

2. **Maximum Units & Integrality**  
   $$
   0 \leq x_j \leq M_j \quad \text{and} \quad x_j \in \mathbb{Z} \quad \forall j
   $$
   (where $M_j$ = maximum allowable units of facility $j$)

3. **Mandatory Facilities**  
   $$
   x_6 \geq 1 \quad \text{(toilet)} \quad \text{and} \quad x_7 \geq 1 \quad \text{(security division)}
   $$

---

## Solution Approach

This is an **integer linear programming (ILP)** problem

- Solved using **branch-and-bound** (or branch-and-cut) in **Gurobi**.
- **Integrality constraints** prevent fractional solutions — cannot simply scale variables proportionally.
- The solver must explore **discrete combinations** of facility units to find the optimal integer solution.

> The optimal integer solution will have **profit ≤ LP relaxation** (Question 4).

In [17]:
Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)

###############################################

# Create a new model
model = gb.Model("Question5")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
# Same setup as Question 4, but with INTEGER variables

# Decision variables: INTEGER
x = model.addVars(Num, vtype=gb.GRB.INTEGER, lb=0, ub=[Max_No[i] for i in range(Num)], name="units")

# Objective: maximize profit
model.setObjective(gb.quicksum(Profit[i] * x[i] for i in range(Num)), gb.GRB.MAXIMIZE)

# Constraints (same as Question 4)
model.addConstr(gb.quicksum(Area[i] * x[i] for i in range(Num)) <= 20000, name="total_area")
model.addConstr(x[6] >= 1, name="min_toilet")
model.addConstr(x[7] >= 1, name="min_security")

model.optimize()

# Print results
print("=" * 60)
print("QUESTION 5: INTEGER VARIABLES")
print("=" * 60)

if model.status == gb.GRB.OPTIMAL:
    print(f"\nMaximum Expected Annual Profit: ${model.ObjVal:,.2f}\n")

    total_area = 0
    print(f"{'Facility':<20} {'Units':<12} {'Profit/Unit':<15} {'Total Profit':<15}")
    print("-" * 65)
    for i in range(Num):
        units = int(x[i].X)
        total_profit = Profit[i] * units
        total_area += Area[i] * units
        if units > 0:
            print(f"{Type[i]:<20} {units:<12} ${Profit[i]:<14,} ${total_profit:<14,.2f}")

    print("-" * 65)
    print(f"\nTotal Occupied Area: {total_area:,} m² (Limit: 20,000 m²)")
else:
    print("Model did not converge")


Set parameter MIPGap to value 1e-05
QUESTION 5: INTEGER VARIABLES

Maximum Expected Annual Profit: $12,475,000.00

Facility             Units        Profit/Unit     Total Profit   
-----------------------------------------------------------------
Restaurant           4            $300,000        $1,200,000.00  
Tech Store           5            $1,000,000      $5,000,000.00  
Pharmacy             4            $500,000        $2,000,000.00  
Dept Store           2            $2,200,000      $4,400,000.00  
Toilet               1            $-25,000        $-25,000.00    
Security             1            $-100,000       $-100,000.00   
-----------------------------------------------------------------

Total Occupied Area: 19,700 m² (Limit: 20,000 m²)


## Question 6: Some Additional Constraints: (20 points)

We are still working based on the setup of Question 5, where we decided to use discrete decision variables. In addition to the two fundamental constraints, we want to maximize the expected profit while adding these four constraints to our model:
1. The number of toilets in the mall should be at least half the number of restaurants.
2. The mall should have at least one complete department store or one complete tech store, or both.
3. The total area the clothing stores occupy should not exceed 5,000 $m^2$.
4. If the mall has more than two complete toy stores, the number of security divisions should be at least two.

Answer the following questions:

### Question 6.1: Mathematical Formulation of the Additional Constraints:

Write the mathematical formulation for each of the above constraints using the techniques you've learned in the course in the following Markdown cell. (Your writing should be clear about which formula is for which one of the additional constraints.) (10 points)

# QUESTION 6: Additional Business Constraints (20 points)

## Additional Constraints

### 1. **Toilet-to-Restaurant Ratio**
$$
x_6 \geq 0.5 \cdot x_0
$$
 **Reasoning**: Ensures adequate sanitation — at least **1 toilet per 2 restaurants**.

---

### 2. **At Least One Premium Store (Department or Tech)**
$$
x_4 + x_2 \geq 1
$$
 **Reasoning**: Requires at least one **anchor tenant** (Department store or Tech store) to attract foot traffic. Implements **OR logic** in selection.

---

### 3. **Clothing Store Area Limit**
$$
1700 \cdot x_1 \leq 5000
\implies x_1 \leq 2.94 \quad \Rightarrow \quad x_1 \leq 2 \quad \text{(since } x_1 \in \mathbb{Z})
$$
**Reasoning**: Prevents over-concentration of clothing stores; promotes **diversity** in store types.

---

### 4. **Conditional Security Requirement**  
 *"If more than 2 toy stores, then at least 2 security units"*

**Binary Indicator Variable**:  
$$
y \in \{0, 1\} \quad \text{where } y = 1 \text{ if } x_5 > 2
$$

**Big-M Formulation** ($M = 5$):
$$
\begin{aligned}
& x_5 \leq 2 + M(1 - y) \\
& x_7 \geq 2 - M(1 - y)
\end{aligned}
$$

**Reasoning**:  
 - If $x_5 > 2$ → $y = 1$ → forces $x_7 \geq 2$  
 - If $x_5 \leq 2$ → $y$ can be 0 → $x_7 \geq 0$ (relaxed)  
 Enforces **safety-driven business rule** using **indicator logic**.

### Question 6.2: Implementing the Additional Constraints:

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a mixed integer programming problem (assume the number of each store/facility is an integer), solve it, and report the results using Gurobi. (10 points)

In [37]:


# Data
Type   = ['Restaurant','Clothing Store','Tech Store','Pharmacy','Dept Store','Toy Store','Toilet','Security']
Profit = [300000,      800000,          1000000,      500000,   2200000,    500000,     -25000,   -100000]
Area   = [500,         1700,            1000,         900,      4300,       1400,       200,      300]
Max_No = [5,           15,              5,            5,        3,          5,          3,        3]
TAREA  = 20000
Num    = len(Type)

# Model
m = gb.Model("Question6")
m.Params.OutputFlag    = 0
m.Params.MIPGap        = 1e-9
m.Params.FeasibilityTol= 1e-9
m.Params.IntFeasTol    = 1e-9

# Integer decision variables with caps
x = m.addVars(Num, vtype=GRB.INTEGER, lb=0, ub=Max_No, name="x")

# Objective
m.setObjective(gb.quicksum(Profit[j]*x[j] for j in range(Num)), GRB.MAXIMIZE)

# Fundamentals
m.addConstr(gb.quicksum(Area[j]*x[j] for j in range(Num)) <= TAREA, name="total_area")
m.addConstr(x[6] >= 1, name="min_toilet")     # Toilet
m.addConstr(x[7] >= 1, name="min_security")   # Security

# Additional constraints
# (1) Toilet-to-restaurant ratio: at least 1 toilet per 2 restaurants
m.addConstr(2*x[6] >= x[0], name="toilet_restaurant_ratio")

# (2) At least one premium store (Dept OR Tech)
m.addConstr(x[4] + x[2] >= 1, name="premium_store")

# (3) Clothing area limit
m.addConstr(1700*x[1] <= 5000, name="clothing_area_limit")

# (4) If Toy > 2 then Security >= 2  (Big-M threshold, tight)
y = m.addVar(vtype=GRB.BINARY, name="toy_ge_3")  # y=1 ⇔ x[5] ≥ 3
M_up = Max_No[5] - 2  # = 3
M_lo = 3              # threshold − lower bound (0)

m.addConstr(x[5] >= 3 - M_lo*(1 - y),     name="toy_thr_lb")  # y=1 → x[5] ≥ 3 ; y=0 → x[5] ≥ 0
m.addConstr(x[5] <= 2 + M_up*y,           name="toy_thr_ub")  # y=0 → x[5] ≤ 2 ; y=1 → x[5] ≤ 5
m.addConstr(x[7] >= 2*y,                  name="security_if_toy")

# Solve
m.optimize()

# Report
obj         = m.ObjVal
units       = [int(x[j].X) for j in range(Num)]
total_area  = sum(Area[j]*units[j]  for j in range(Num))
total_profit= sum(Profit[j]*units[j] for j in range(Num))

print("="*64)
print("QUESTION 6: ADDITIONAL CONSTRAINTS (INTEGER + Big-M)")
print("="*64)
print(f"\nMaximum Expected Annual Profit: ${obj:,.2f}\n")
print(f"{'Facility':<16}{'Units':>8}  {'Profit/Unit':>14}  {'Total Profit':>15}")
print("-"*64)
for j in range(Num):
    print(f"{Type[j]:<16}{units[j]:>8d}  ${Profit[j]:>13,.0f}  ${Profit[j]*units[j]:>14,.2f}")
print("-"*64)
print(f"Total Occupied Area: {total_area:,} m² (Limit: {TAREA:,} m²)")

# Validations (exact booleans)
print("\nValidation:")
print("  Fundamental area:", total_area <= TAREA)
print("  Toilets ≥ 0.5·Restaurants:", 2*units[6] >= units[0])
print("  Premium store present:", units[4] + units[2] >= 1)
print("  Clothing area ≤ 5000:", 1700*units[1] <= 5000)
print("  Toy>2 ⇒ Security≥2 :", (units[5] <= 2) or (units[7] >= 2))


QUESTION 6: ADDITIONAL CONSTRAINTS (INTEGER + Big-M)

Maximum Expected Annual Profit: $12,450,000.00

Facility           Units     Profit/Unit     Total Profit
----------------------------------------------------------------
Restaurant             4  $      300,000  $  1,200,000.00
Clothing Store         0  $      800,000  $          0.00
Tech Store             5  $    1,000,000  $  5,000,000.00
Pharmacy               4  $      500,000  $  2,000,000.00
Dept Store             2  $    2,200,000  $  4,400,000.00
Toy Store              0  $      500,000  $          0.00
Toilet                 2  $      -25,000  $    -50,000.00
Security               1  $     -100,000  $   -100,000.00
----------------------------------------------------------------
Total Occupied Area: 19,900 m² (Limit: 20,000 m²)

Validation:
  Fundamental area: True
  Toilets ≥ 0.5·Restaurants: True
  Premium store present: True
  Clothing area ≤ 5000: True
  Toy>2 ⇒ Security≥2 : True


## Question 7: The Case for Diminishing Returns: (15 points)

**Diminishing return to scale**: We expect the marginal profit of each additional unit from each store to decrease. That is, while a single pharmacy in the mall brings about \$500,000 annual profit, having two pharmacies in the mall creates \$800,000, and three pharmacies make \$900,000 together. Therefore, the marginal profit of adding a new pharmacy is \$500,000 when there is no pharmacy, \$300,000 when there is one pharmacy, and \$100,000 when there are already two pharmacies in the mall.

Considering this phenomenon, our expectation for the annual profit created by the different number of restaurants, technology stores, and pharmacies is updated as in the following table: (Do not forget that the numbers in the following table represent the profit we expect all the stores to create together, not the profit produced by each of them. So, for example, we expect three restaurants to make \$750,000 altogether.)

| Store | Profit of one unit ($) | Two units | Three units | Four units | Five units |
| :--: | :--: | :--: | :--: | :--: | :--: |
| Restaurant | 300,000 | 550,000 | 750,000 | 850,00 | 900,000 |
| Tech Store | 1,000,000 | 1,750,000 | 2,250,000 | 2,500,000 | 2,600,000 |
| Pharmacy | 500,000 | 800,000 | 900,000 | 950,000 | 950,000 |

The maximum number limit for each type of store/facility remains the same as the limits depicted in the original table, and the expected profit for stores/facilities other than restaurants, technology stores, and pharmacies is estimated linearly similar to the previous questions.

For example, suppose we decided to have 2 restaurants, 2 clothing stores, 1 tech store, 3 pharmacies, 3 toy stores, 1 toilet, and 2 security divisions in the mall. Then, the expected profit of this decision is:
$$\$550,000 + 2 \times \$800,000 + \$1,000,000 + \$900,000 + 3 \times \$500,000 - 1 \times \$25,000 - 2 \times \$100,000 = \$5,325,000$$

In addition to diminishing return to scale and the two fundamental constraints, we still want to entertain the constraints we introduced in Question 6.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a mixed integer programming problem (assume the number of each store/facility is an integer), solve it, and report the results using Gurobi.

# Diminishing Returns to Scale – Full Integer Model with Step Profits

> **For Restaurants (R), Tech (T), and Pharmacy (P)**, total profit is **cumulative and level-dependent** (diminishing marginal returns).  
> For all other facilities, profit remains **linear**.

---

## Decision Variables

- $ b_{j,k} \in \{0,1\} $ for $ j \in \{R, T, P\}, \, k = 0, \dots, 5 $  
  → $ b_{j,k} = 1 $ if **exactly $ k $ units** of facility $ j $ are chosen.

- $ x_R, x_T, x_P \in \mathbb{Z}_{\geq 0} $ with $ x_j \leq 5 $  
  $$
  x_j = \sum_{k=0}^{5} k \cdot b_{j,k} \quad \text{for } j \in \{R, T, P\}
  $$

- $ x_C \in \mathbb{Z}_{\geq 0}, \, x_C \leq 15 $ (Clothing)  
- $ x_D \in \mathbb{Z}_{\geq 0}, \, x_D \leq 3 $ (Department)  
- $ x_Y \in \mathbb{Z}_{\geq 0}, \, x_Y \leq 5 $ (Toy)  
- $ x_L \in \mathbb{Z}_{\geq 0}, \, x_L \leq 3 $ (Toilet)  
- $ x_S \in \mathbb{Z}_{\geq 0}, \, x_S \leq 3 $ (Security)

- $ y \in \{0,1\} $: Toy threshold indicator  
  → $ y = 1 $ if $ x_Y \geq 3 $

---

## "Choose-One" Level Selection (for R/T/P)

$$
\sum_{k=0}^{5} b_{j,k} = 1 \quad \forall j \in \{R, T, P\}
\qquad
x_j = \sum_{k=0}^{5} k \cdot b_{j,k}
$$

---

## Objective (Maximize Expected Annual Profit)

Let $ P_{R,k}^{\text{tot}}, P_{T,k}^{\text{tot}}, P_{P,k}^{\text{tot}} $ be the **table totals** for $ k $ units (from the prompt).

Linear profits for others:  
$ p_C = 800{,}000, \quad p_D = 2{,}200{,}000, \quad p_Y = 500{,}000, \quad p_L = -25{,}000, \quad p_S = -100{,}000 $

$$
\max \;
\sum_{k=0}^{5} P_{R,k}^{\text{tot}} b_{R,k} +
\sum_{k=0}^{5} P_{T,k}^{\text{tot}} b_{T,k} +
\sum_{k=0}^{5} P_{P,k}^{\text{tot}} b_{P,k} +
800{,}000 x_C +
2{,}200{,}000 x_D +
500{,}000 x_Y -
25{,}000 x_L -
100{,}000 x_S
$$

---

## Fundamental Constraints

$$
500 x_R + 1700 x_C + 1000 x_T + 900 x_P + 4300 x_D + 1400 x_Y + 200 x_L + 300 x_S \leq 20{,}000
$$

$$
x_L \geq 1, \quad x_S \geq 1
$$

---

## Additional Business Constraints (from Q6)

1. **Toilets vs. Restaurants**:  
   $$
   2 x_L \geq x_R
   $$

2. **At least one premium (Dept or Tech)**:  
   $$
   x_D + x_T \geq 1
   $$

3. **Clothing area cap**:  
   $$
   1700 x_C \leq 5{,}000 \quad \Rightarrow \quad x_C \leq 2 \text{ (since integer)}
   $$

4. **If Toy > 2 → Security ≥ 2**:  
   $$
   \begin{aligned}
   x_Y &\leq 2 + 3y \\
   x_S &\geq 2 - 3(1 - y) \\
   y &\in \{0,1\}
   \end{aligned}
   $$

---


In [39]:
# Q7 — Diminishing Returns (MIP) with Q6 constraints

import gurobipy as gb
from gurobipy import GRB

# Indices: 0=Restaurant, 1=Clothing, 2=Tech, 3=Pharmacy, 4=Dept, 5=Toy, 6=Toilet, 7=Security
labels = ['Restaurant','Clothing','Tech','Pharmacy','Dept','Toy','Toilet','Security']

# Areas, caps
A = [500, 1700, 1000, 900, 4300, 1400, 200, 300]
U = [5,   15,   5,    5,   3,    5,    3,   3]
TAREA = 20000

# Linear profits for non-R/T/P
P_lin = [0, 800000, 0, 0, 2200000, 500000, -25000, -100000]

# Diminishing-returns TOTAL profit tables (k = 0..5 units)
Rest_tot = [0, 300000,  550000,  750000,  850000,  900000]
Tech_tot = [0, 1000000, 1750000, 2250000, 2500000, 2600000]
Phar_tot = [0, 500000,  800000,  900000,  950000,  950000]

m = gb.Model("Q7_dim_returns")
m.Params.OutputFlag = 0
m.Params.MIPGap = 1e-9
m.Params.FeasibilityTol = 1e-9
m.Params.IntFeasTol = 1e-9

# --- Level-selection binaries for R/T/P ---
bR = m.addVars(6, vtype=GRB.BINARY, name="bR")  # exactly one chosen
bT = m.addVars(6, vtype=GRB.BINARY, name="bT")
bP = m.addVars(6, vtype=GRB.BINARY, name="bP")
m.addConstr(gb.quicksum(bR[k] for k in range(6)) == 1)
m.addConstr(gb.quicksum(bT[k] for k in range(6)) == 1)
m.addConstr(gb.quicksum(bP[k] for k in range(6)) == 1)

# Integer counts for all types
xR = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[0], name="xR")
xC = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[1], name="xC")
xT = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[2], name="xT")
xP = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[3], name="xP")
xD = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[4], name="xD")
xY = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[5], name="xY")
xL = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[6], name="xL")
xS = m.addVar(vtype=GRB.INTEGER, lb=0, ub=U[7], name="xS")

# Bind R/T/P counts to chosen level
m.addConstr(xR == gb.quicksum(k * bR[k] for k in range(6)))
m.addConstr(xT == gb.quicksum(k * bT[k] for k in range(6)))
m.addConstr(xP == gb.quicksum(k * bP[k] for k in range(6)))

# Fundamentals
m.addConstr(A[0]*xR + A[1]*xC + A[2]*xT + A[3]*xP + A[4]*xD + A[5]*xY + A[6]*xL + A[7]*xS <= TAREA)
m.addConstr(xL >= 1)
m.addConstr(xS >= 1)

# Q6 additional constraints
m.addConstr(2*xL >= xR)                 # toilet-to-restaurant ratio
m.addConstr(xD + xT >= 1)               # at least one premium (Dept or Tech)
m.addConstr(1700*xC <= 5000)            # clothing area cap

# Toy > 2 ⇒ Security ≥ 2  (tight threshold with one binary)
y = m.addVar(vtype=GRB.BINARY, name="toy_ge_3")
m.addConstr(xY >= 3*y)
m.addConstr(xY <= 2 + (U[5]-2)*y)       # ensures y=0 ↔ xY≤2, y=1 ↔ xY≥3..5
m.addConstr(xS >= 2*y)

# Objective: totals for R/T/P + linear for others
obj = (gb.quicksum(Rest_tot[k]*bR[k] for k in range(6)) +
       gb.quicksum(Tech_tot[k]*bT[k] for k in range(6)) +
       gb.quicksum(Phar_tot[k]*bP[k] for k in range(6)) +
       P_lin[1]*xC + P_lin[4]*xD + P_lin[5]*xY + P_lin[6]*xL + P_lin[7]*xS)
m.setObjective(obj, GRB.MAXIMIZE)

m.optimize()

# --- Output ---
profit = m.ObjVal
area = (A[0]*xR.X + A[1]*xC.X + A[2]*xT.X + A[3]*xP.X + A[4]*xD.X + A[5]*xY.X + A[6]*xL.X + A[7]*xS.X)

print("QUESTION 7: DIMINISHING RETURNS (MIP)")
print(f"Max expected annual profit: ${profit:,.2f}")
print(f"Total occupied area: {area:.0f} m² (limit {TAREA})")
print(f"{'Facility':<12}{'Units':>8}")
print("-"*22)
print(f"{'Restaurant':<12}{int(xR.X):>8d}")
print(f"{'Clothing':<12}{int(xC.X):>8d}")
print(f"{'Tech':<12}{int(xT.X):>8d}")
print(f"{'Pharmacy':<12}{int(xP.X):>8d}")
print(f"{'Dept':<12}{int(xD.X):>8d}")
print(f"{'Toy':<12}{int(xY.X):>8d}")
print(f"{'Toilet':<12}{int(xL.X):>8d}")
print(f"{'Security':<12}{int(xS.X):>8d}")

# Validations
assert area <= TAREA + 1e-6
assert 2*int(xL.X) >= int(xR.X)
assert int(xD.X) + int(xT.X) >= 1
assert 1700*int(xC.X) <= 5000
assert (int(xY.X) <= 2) or (int(xS.X) >= 2)


QUESTION 7: DIMINISHING RETURNS (MIP)
Max expected annual profit: $10,575,000.00
Total occupied area: 20000 m² (limit 20000)
Facility       Units
----------------------
Restaurant         2
Clothing           1
Tech               3
Pharmacy           1
Dept               3
Toy                0
Toilet             1
Security           1


# Problem 3: BlancedMilk Needs Your Help (Again!)

We have been through the problems BalancedMilk faces after it lost its largest supplier multiple times. (You may review Assignment and Quiz 1 if you forgot them.) The issue is not resolved yet, and $S_8$ is still shut down. BalancedMilk managers are so excited that now you know how to implement logical constraints using MIP, because they want you to help them with their new problem! They gave you the following information:
1. BalancedMilk is **not** committed to distributing the milk supplied by the remaining supply centers, $S_1$ to $S_7$. (It has the option not to distribute the milk it receives from the remaining supply centers.) Additionally, it **must not** oversupply any demand market.
2. The cost of not supplying (not delivering) milk to each demand market is 15$/tonne.
3. $S_3$, $S_4$, and $S_5$ belong to the same company. The company has imposed the following constraint: BalancedMilk should utilize $\geq50\%$ (equal or more than $50\%$) of the capacity of at least two of the three supply centers owned by this company.
4. Because $D_5$ and $D_6$ are close, the firm has some flexibility in answering their demands. So either $D_5$ receives $\geq80\%$ of its demand, or $D_6$ receives $\geq60\%$ of its demand, or both.
5. A competitor is trying to damage BalancedMilk's reputation by focusing on the discrimination it imposes between different demand centers. This competitor is trying to exploit the difference between $D_2$ and $D_8$ and $D_9$. Therefore, to respond to this criticism, if $D_2$ receives $> 50\%$ of its demand, then $D_8$ and $D_9$ should both receive $\geq50\%$ of their demand as well.

(Trust me. I am not a recruiter for any dairy transportation company, and neither am I preparing the MMA cohort for an internship at one. It is just that BalancedMilk has so many unresolved issues!)

## Question 8: The formulation (15 points)
Write the mathematical formulation of this problem using the techniques you've learned in the course in the following Markdown cell. Use the variables we have already defined.

$x_{ij}$ shows the amount of milk (in tonnes) transported from supply center $i$ to demand center $j$. \
$c_{ij}$ is the transportation cost from supply center $i$ to demand center $j$. \
$S_i$ is the capacity of supply center $i$. \
$D_j$ is the demand of demand center $j$.

We introduce the following auxiliary variables: \
$y_i$ shows the amount of milk (in tonnes) supplied by supply center $i$: $y_i = \sum_{j = 1}^{10} x_{ij} \; for \; i = 1, \dots, 7.$ \
$z_j$ shows the amount of milk (in tonnes) delivered to demand center $j$. $z_j = \sum_{i = 1}^7 x_{ij} \; for \; j = 1, \dots, 10$.

**The objective function is**:


**The constraints are**: \
**Supply and demand constraints:** \


Logical constraints: \
**First logical constraint:**


**Second logical constraint:**


**Third logical constraint:**



# BalancedMilk (Q8) — Mathematical Formulation

---

## Sets & Indices
- **Supply centers**: $ i \in \{1, \dots, 7\} $ → $ (S_1, \dots, S_7) $
- **Demand centers**: $ j \in \{1, \dots, 10\} $ → $ (D_1, \dots, D_{10}) $

---

## Data
- $ S_i $ = capacity of supply center $ i $ (tonnes)  
- $ D_j $ = demand of center $ j $ (tonnes)  
- $ c_{ij} $ = transport cost per tonne from $ i $ to $ j $  
- **Penalty for unmet demand**: **15 $/tonne**  
- **Big-M**: $ M := \max_j D_j $ (safe and tight enough)  
- **Small ε**: $ \epsilon = 10^{-6} $ (for strict “>” when needed)

---

## Decision Variables
- $ x_{ij} \geq 0 $: **tonnes shipped** from $ i $ to $ j $  
- $ u_j \geq 0 $: **unmet demand** at $ D_j $  
- $ b_3, b_4, b_5 \in \{0,1\} $:  
  → $ b_k = 1 \Rightarrow S_k $ uses **≥ 50%** of its capacity ($ k = 3,4,5 $)  
- $ d_5, d_6 \in \{0,1\} $:  
  → $ d_5 = 1 \Rightarrow D_5 $ gets **≥ 80%**  
  → $ d_6 = 1 \Rightarrow D_6 $ gets **≥ 60%**  
- $ y_2 \in \{0,1\} $: **trigger** for “$ D_2 > 50\% $ served”

---

## Shorthand (Defined by Constraints)
- Total **out** of supply $ i $:  
  $$
  y_i := \sum_{j=1}^{10} x_{ij}
  $$
- Total **into** demand $ j $:  
  $$
  z_j := \sum_{i=1}^{7} x_{ij}
  $$

---

## Objective
$$
\min \;
\sum_{i=1}^{7} \sum_{j=1}^{10} c_{ij} x_{ij} \;+\; 15 \sum_{j=1}^{10} u_j
$$

---

## Core Constraints

### 1. **Supply Capacity**
$$
\sum_{j=1}^{10} x_{ij} \leq S_i \quad \forall i = 1,\dots,7
$$

### 2. **Demand Balance with Shortage (no oversupply)**
$$
\sum_{i=1}^{7} x_{ij} + u_j = D_j \quad \forall j = 1,\dots,10
$$



---
## Logical Constraints

---

### A) **Flexibility**: At least one of $ D_5 $ (≥80%) **or** $ D_6 $ (≥60%)

$$
\begin{aligned}
z_5 &\geq 0.8 D_5 - M(1 - d_5) \\
z_6 &\geq 0.6 D_6 - M(1 - d_6) \\
d_5 + d_6 &\geq 1
\end{aligned}
$$

---

### B) **Company Policy**: At least **two** of $ S_3, S_4, S_5 $ use **≥50% capacity**

$$
\begin{aligned}
y_3 &\geq 0.5 S_3 - M(1 - b_3) \\
y_4 &\geq 0.5 S_4 - M(1 - b_4) \\
y_5 &\geq 0.5 S_5 - M(1 - b_5) \\
b_3 + b_4 + b_5 &\geq 2
\end{aligned}
$$

---

### C) **Anti-Discrimination**:  
> **If $ D_2 $ is more than 50% served**, then **$ D_8 $ and $ D_9 $ must be ≥50%**

*(Two-sided link for the trigger + enforced consequences)*

$$
\begin{aligned}
z_2 - 0.5 D_2 &\geq \epsilon - M(1 - y_2) \\
z_2 - 0.5 D_2 &\leq M y_2 \\
z_8 &\geq 0.5 D_8 - M(1 - y_2) \\
z_9 &\geq 0.5 D_9 - M(1 - y_2)
\end{aligned}
$$

> **Logic**:  
> - $ z_2 > 0.5 D_2 \Rightarrow y_2 = 1 \Rightarrow z_8, z_9 \geq 0.5 D $  
> - $ z_2 \leq 0.5 D_2 \Rightarrow y_2 = 0 $ → constraints relaxed

---

## Variable Domains

$$
x_{ij} \geq 0, \quad u_j \geq 0, \quad b_3, b_4, b_5, d_5, d_6, y_2 \in \{0,1\}
$$

---


- **Unmet demand** modeled correctly via $ u_j $ with equality:  
  $$
  z_j + u_j = D_j
  $$


## Question 9: The implementation (10 points)

Write the code for this problem using Gurobi and see if an optimal solution with these constraints exists. If yes, what are the lowest cost, the utilized capacity of each supply center, and the satisfied demand of each demand center?

(Remember that while the indices of suppliers and demand markets start from 1, the indices of the variables in Python begin from 0.)

In [44]:


Cost = [
    [20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
    [15, 53,  7, 20, 47, 11, 16, 17, 15, 44],
    [22, 25, 42, 22, 31,  9, 11, 29, 20,  5],
    [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
    [ 9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
    [21, 24, 32, 49,  5, 47, 30, 23, 37,  8],
    [12, 19,  5, 28, 47, 39, 15, 35,  9, 51]
]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]

DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

# -------------------------------
# CREATE MODEL
# -------------------------------
model = gb.Model("Question9")
model.Params.OutputFlag = 0
model.setParam('MIPGap', 0.00001)

# -------------------------------
# DECISION VARIABLES
# -------------------------------
x = model.addVars(n_supply, n_demand, vtype=GRB.CONTINUOUS, lb=0, name="x")  # transport tonnes
u = model.addVars(n_demand, vtype=GRB.CONTINUOUS, lb=0, name="u")             # unmet demand (shortage)

# Binaries for logical conditions
b = model.addVars(3, vtype=GRB.BINARY, name="b_S3S4S5")  # S3,S4,S5 half capacity flags
d5 = model.addVar(vtype=GRB.BINARY, name="d5_ge80")      # D5 >= 80%
d6 = model.addVar(vtype=GRB.BINARY, name="d6_ge60")      # D6 >= 60%
y2 = model.addVar(vtype=GRB.BINARY, name="D2_gt50pct")   # trigger for D2 > 50%

# -------------------------------
# OBJECTIVE FUNCTION
# -------------------------------
# minimize total transport cost + unmet penalty (15 per tonne)
model.setObjective(
    gb.quicksum(Cost[i][j] * x[i, j] for i in Range_supply for j in Range_demand) +
    15.0 * gb.quicksum(u[j] for j in Range_demand),
    GRB.MINIMIZE
)

# -------------------------------
# FUNDAMENTAL CONSTRAINTS
# -------------------------------
# Supply ≤ capacity
for i in Range_supply:
    model.addConstr(gb.quicksum(x[i, j] for j in Range_demand) <= Supply[i], name=f"supply_cap[{i+1}]")

# Demand balance: served + unmet = demand
for j in Range_demand:
    model.addConstr(gb.quicksum(x[i, j] for i in Range_supply) + u[j] == Demand[j], name=f"demand_bal[{j+1}]")

# -------------------------------
# LOGICAL CONSTRAINTS (Q8)
# -------------------------------
M = max(max(Demand), max(Supply))
eps = 1e-6

# 1) FLEXIBILITY: D5 ≥ 80% OR D6 ≥ 60%
j5, j6 = 4, 5
model.addConstr(gb.quicksum(x[i, j5] for i in Range_supply) >= 0.8 * Demand[j5] - M * (1 - d5))
model.addConstr(gb.quicksum(x[i, j6] for i in Range_supply) >= 0.6 * Demand[j6] - M * (1 - d6))
model.addConstr(d5 + d6 >= 1, name="flexibility_or")

# 2) COMPANY POLICY: At least 2 of S3,S4,S5 use ≥ 50% capacity
for idx, si in enumerate([2, 3, 4]):  # zero-based S3,S4,S5
    model.addConstr(gb.quicksum(x[si, j] for j in Range_demand) >= 0.5 * Supply[si] - M * (1 - b[idx]),
                    name=f"S{si+1}_half_or_relaxed")
model.addConstr(b.sum() >= 2, name="two_of_three_half")

# 3) ANTI-DISCRIMINATION: If D2 > 50% → D8, D9 ≥ 50%
j2, j8, j9 = 1, 7, 8
z2 = gb.quicksum(x[i, j2] for i in Range_supply)
z8 = gb.quicksum(x[i, j8] for i in Range_supply)
z9 = gb.quicksum(x[i, j9] for i in Range_supply)

model.addConstr(z2 - 0.5 * Demand[j2] >= eps - M * (1 - y2), name="D2_trigger_lb")
model.addConstr(z2 - 0.5 * Demand[j2] <= M * y2,            name="D2_trigger_ub")
model.addConstr(z8 >= 0.5 * Demand[j8] - M * (1 - y2),      name="D8_ge50_if_D2_half")
model.addConstr(z9 >= 0.5 * Demand[j9] - M * (1 - y2),      name="D9_ge50_if_D2_half")

# -------------------------------
# OPTIMIZE MODEL
# -------------------------------
model.optimize()

# -------------------------------
# RESULTS
# -------------------------------
if model.status == GRB.OPTIMAL:
    print("=" * 70)
    print("BalancedMilk Optimization")
    print("=" * 70)
    print(f"Total Cost (Transport + Unmet Penalties): ${model.ObjVal:,.2f}\n")

    shipped = np.array([[x[i, j].X for j in Range_demand] for i in Range_supply])
    unmet = np.array([u[j].X for j in Range_demand])
    used_S = shipped.sum(axis=1)
    served = shipped.sum(axis=0)

    print("SUPPLY UTILIZATION (tonnes):")
    for i in Range_supply:
        print(f"  {SupplyCenter[i]}: {used_S[i]:.2f} / {Supply[i]}")

    print("\nDEMAND SERVED (tonnes):")
    for j in Range_demand:
        print(f"  {DemandMarket[j]}: served {served[j]:.2f} , unmet {unmet[j]:.2f} , demand {Demand[j]}")

    # Logical validations
    print("\nVALIDATION OF LOGICAL CONSTRAINTS:")
    print(f"  D5≥80%? {served[j5] >= 0.8 * Demand[j5]:}  |  D6≥60%? {served[j6] >= 0.6 * Demand[j6]:}")
    half_flags = [used_S[2] >= 0.5 * Supply[2], used_S[3] >= 0.5 * Supply[3], used_S[4] >= 0.5 * Supply[4]]
    print(f"  # of S3,S4,S5 at ≥50%: {sum(half_flags)}  (S3={half_flags[0]}, S4={half_flags[1]}, S5={half_flags[2]})")
    if served[j2] > 0.5 * Demand[j2]:
        print(f"  D2>50% triggered → D8≥50%={served[j8] >= 0.5*Demand[j8]}, D9≥50%={served[j9] >= 0.5*Demand[j9]}")
else:
    print("Model did not converge to optimality.")




BalancedMilk Optimization
Total Cost (Transport + Unmet Penalties): $15,860.00

SUPPLY UTILIZATION (tonnes):
  S_1: 110.00 / 110
  S_2: 80.00 / 80
  S_3: 100.00 / 100
  S_4: 100.00 / 240
  S_5: 100.00 / 100
  S_6: 250.00 / 280
  S_7: 130.00 / 130

DEMAND SERVED (tonnes):
  D_1: served 90.00 , unmet 0.00 , demand 90
  D_2: served 100.00 , unmet 0.00 , demand 100
  D_3: served 150.00 , unmet 0.00 , demand 150
  D_4: served 0.00 , unmet 190.00 , demand 190
  D_5: served 180.00 , unmet 0.00 , demand 180
  D_6: served 115.00 , unmet 125.00 , demand 240
  D_7: served 0.00 , unmet 210.00 , demand 210
  D_8: served 45.00 , unmet 45.00 , demand 90
  D_9: served 120.00 , unmet 40.00 , demand 160
  D_10: served 70.00 , unmet 0.00 , demand 70

VALIDATION OF LOGICAL CONSTRAINTS:
  D5≥80%? True  |  D6≥60%? False
  # of S3,S4,S5 at ≥50%: 2  (S3=True, S4=False, S5=True)
  D2>50% triggered → D8≥50%=True, D9≥50%=True
